In [2]:
# imports and setup

from solid import (
    circle,
    cylinder,
    translate,
    cube,
    rotate,
    rotate_extrude,
    square,
    polygon,
)
import viewscad
r = viewscad.Renderer()


In [3]:
# main dimensions
thick = 2
lip_thick = .5
screw_od = 3.5

# system parameter
weight = {
    "l": 32.3,
    "w": 19,
    "h":3,
    "weight": .5 #oz
}

bearing = {
    "od":23,
    "id": 7.75,
    "odr":18.5,
    "idr": 11.5,
    "odd":27,
    "h":7,
    "weight":.5
}

# balance arms...
arm = {"l1":45, "l2":90, "l3":25, "l4":150}



In [4]:
# flat platform to mount acc, motor, weights to dowel...
dowel_od = 7.5
dowel_tr = 10

flat_l, flat_w = 75, 50

# flat cube...
flat_base = cube([flat_l, flat_w, thick], center=True)
flat_base = translate([0, 0, thick/2])(flat_base)


dowel_rect = cube([20, dowel_tr, dowel_tr], center=True)
dowel_rect = translate([flat_l/2-5, 0, dowel_tr/2])(dowel_rect)

dowel_hole = cylinder(d=dowel_od, h=20)
dowel_hole = rotate([0,90,0])(dowel_hole)
dowel_hole = translate([flat_l/2-5, 0, dowel_tr/2])(dowel_hole)

screw_hole = cylinder(d=4, h=20)
screw_hole = translate([flat_l/2-5, 0, 0])(screw_hole)


p = flat_base+dowel_rect-dowel_hole-screw_hole

r.render(
        p,
        file_header="$fa=.01;\n $fs=0.01",
        outfile=f"STL/FlatMount.stl",
    )


In [5]:
# Arm with flat pieces

bearing_tube = cylinder(d=bearing["odd"], h=bearing["h"])
bearing_ledge = cylinder(d=bearing["idr"], h=thick)

bearing_center = bearing_tube+bearing_ledge     # +screw_tube+screw_tube_2
screw_p = translate([0,-bearing["odd"]/2,bearing["h"]/2])(cube([screw_od*2, 2*thick, bearing["h"]], center=True))

flat_h=3
a1_1 = cube([arm["l1"], thick, flat_h], center=True)
a1_1 = translate([arm["l1"]/2, screw_od/2+thick/2, flat_h/2])(a1_1)
a1_2 = translate([0, -(screw_od+thick), 0])(a1_1)

# flat cube...
flat_base = cube([2*weight["w"], weight["l"], flat_h], center=True)
flat_base = translate([arm["l1"]+2*weight["w"]/2-5, 0, flat_h/2])(flat_base)

a1 = a1_1+a1_2+flat_base


# second arm with bearing extension
a2_1 = cube([arm["l2"], thick, bearing["h"]], center=True)
a2_1 = translate([arm["l2"]/2, screw_od/2+thick/2, bearing["h"]/2])(a2_1)
a2_2 = translate([0, -(screw_od+thick), 0])(a2_1)
bearing_offset = bearing["h"]+6*thick
a2_ext_base =cylinder(d=bearing["idr"], h=bearing_offset)
a2_ext_bearing = translate([0,0,bearing_offset])(cylinder(d=bearing["id"], h=bearing["h"]/2))
a2_ext = translate([arm["l2"],0,0])(a2_ext_base+a2_ext_bearing)
a2_h = translate([arm["l2"], 0, 0])(cylinder(d=4, h=50))    # hole through center of bearing connection

a2 = a2_1+a2_2+a2_ext-a2_h
a2 = rotate([0,0,180])(a2)

# holes ---------------------------
bearing_hole = translate([0,0,lip_thick])(cylinder(d=bearing["od"], h=bearing["h"]))
bearing_hole_through = cylinder(d=bearing["odr"], h=lip_thick)
screw_hole = translate([0,0,(bearing["h"]+lip_thick)/2])(rotate([90,0,0])(cylinder(d=3, h=50)))

holes = bearing_hole+bearing_hole_through+screw_hole

p = bearing_center+screw_p+a1+a2-holes

r.render(
        p,
        file_header="$fa=.01;\n $fs=0.01",
        outfile=f"STL/P1.stl",
    )


In [6]:
# mount for accelerometer
dowel_d = 40
dowel_odd = 10
dowel_od = 7.6

flat = cube([dowel_d, dowel_d, thick], center=True)
flat = translate([0,0,thick/2])(flat)

dowel_cube = cube([dowel_d, dowel_odd, dowel_odd], center=True)
dowel_cube_1 = translate([0,dowel_d/2,dowel_odd/2])(dowel_cube)
dowel_cube_2 = translate([0,-dowel_d/2,dowel_odd/2])(dowel_cube)
dc = dowel_cube_1+dowel_cube_2

dowel_hole = cylinder(d=dowel_od, h=30)
dowel_hole = rotate([0,90,0])(dowel_hole)
dowel_hole_1 = translate([0,dowel_d/2,dowel_odd/2])(dowel_hole)
dowel_hole_2 = translate([0,-dowel_d/2,dowel_odd/2])(dowel_hole)
dh = dowel_hole_1+dowel_hole_2
dh += rotate([0,0,180])(dh)

screw_hole = cylinder(d=3, h=30)
screw_hole = rotate([90,0,0])(screw_hole)
screw_hole = translate([0,0,dowel_od/2])(screw_hole)
screw_hole += rotate([0,0,180])(screw_hole)


p = flat+dc-dh-screw_hole

r.render(
        p,
        file_header="$fa=.01;\n $fs=0.01",
        outfile=f"STL/D_Flat.stl",
    )



TraitError: The 'target' trait of a DirectionalLight instance expected an Uninitialized or an Object3D, not the str 'IPY_MODEL_9861b69c-41a8-46ad-8ccf-08688db59962'.

In [7]:

# motor cutouts
mt = cylinder(d=34, h=6)
motor_hole = cylinder(d=30, h=30)
motor_slot = cube([dowel_d, thick, 10], center=True)
motor_slot = translate([dowel_d/2,0,thick/2])(motor_slot)

p = flat+mt+dc-dh-motor_hole-motor_slot-screw_hole

r.render(
        p,
        file_header="$fa=.01;\n $fs=0.01",
        outfile=f"STL/D_Motor.stl",
    )

TraitError: The 'target' trait of a DirectionalLight instance expected an Uninitialized or an Object3D, not the str 'IPY_MODEL_9e5dee5e-635e-4d37-bced-48831a5d88c6'.

In [8]:
# dowel bearing attachment
flat = cube([dowel_d, dowel_d, bearing['h']], center=True)
flat = translate([0,0,bearing['h']/2])(flat)

bearing_hole = cylinder(d=bearing['od'], h=bearing['h'])
bearing_hole = translate([0,0,thick])(bearing_hole)

bearing_through_hole = cylinder(d=bearing['odr'], h=30)


p = flat+dc-dh-bearing_hole-bearing_through_hole-screw_hole

r.render(
        p,
        file_header="$fa=.01;\n $fs=0.01",
        outfile=f"STL/Bearing_Dowel.stl",
    )

TraitError: The 'target' trait of a DirectionalLight instance expected an Uninitialized or an Object3D, not the str 'IPY_MODEL_5c7572c0-8f3b-4091-91cf-e2f2a079f578'.

TraitError: The 'target' trait of a DirectionalLight instance expected an Uninitialized or an Object3D, not the str 'IPY_MODEL_c51ff9fc-c7c3-48df-ba27-85db00987420'.

In [21]:
# dowel bearing attachment
shaft = {'c': 7, 'hole':2.5, 'height':10}
arm = {'l': 30, 'w':4}
thick=2
weight = {
    "l": 32.3,
    "w": 19,
    "h":3,
    "weight": .5 #oz
}

shaft_c = cube([shaft['c'], shaft['c'], shaft['height']], center=True)
shaft_c = translate([0,0,10/2])(shaft_c)

shaft_hole = cylinder(d=shaft['hole'], h=shaft['height'])

shaft_set = cylinder(d=2, h=shaft['height'])
shaft_set = rotate([0,90,0])(shaft_set)
shaft_set = translate([0,0,4])(shaft_set)
shaft_set += translate([0,0,3])(shaft_set)


arm_w = cube([arm['l'], arm['w'], thick], center=True)
arm_w = translate([arm['l']/2,0,thick/2])(arm_w)
arm_flat = cube([weight['l'], weight['w'], thick], center=True)
arm_flat = translate([arm['l']+weight['l']/2,0,thick/2])(arm_flat)

arm_w_0 = arm_w+arm_flat
arm_w_1 = rotate([0,0,120])(arm_w_0)
arm_w_2 = rotate([0,0,240])(arm_w_0)
arm_weights = arm_w_0 + arm_w_1 + arm_w_2

p = shaft_c+arm_weights-shaft_hole-shaft_set

r.render(p, file_header="$fa=.01;\n $fs=0.01", outfile=f"STL/ShaftMount.stl")

TraitError: The 'target' trait of a DirectionalLight instance expected an Uninitialized or an Object3D, not the str 'IPY_MODEL_fdf23364-e1dd-404c-a4ef-1648cf08d077'.